In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")
from postgresRDSConnect import host, database, user, password, port

In [ ]:
# Establishing a connection
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connection to PostgreSQL database successful")
except psycopg2.Error as e:
    print("Error: Could not connect to PostgreSQL database")
    print(e)

In [5]:
query = 'SELECT * FROM "evChargers".evchargesupdated_v2;'

In [6]:
df = pd.read_sql_query(query, conn)

C:\Users\alawr\AppData\Local\Temp\ipykernel_31304\1553155693.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [9]:
df.columns

Index(['station name', 'street address', 'city', 'state', 'zipcode', 'country',
       'station phone', 'groups with access code', 'access days time',
       'charger count', 'ev other info', 'ev network', 'latitude', 'longitude',
       'date last confirmed', 'ev connector types', 'facility type',
       'ev pricing', 'geom'],
      dtype='object')

In [10]:
nj_rows = df[df['state'] == 'NJ']
nj_rows['zipcode'] = nj_rows['zipcode'].apply(lambda x: x.zfill(5))
df.update(nj_rows)

C:\Users\alawr\AppData\Local\Temp\ipykernel_31304\3624520606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nj_rows['zipcode'] = nj_rows['zipcode'].apply(lambda x: x.zfill(5))


In [16]:
cleaned_df = df[df['facility type'] != '']
cleaned_df.head()

,station name,street address,city,state,zipcode,country,station phone,groups with access code,access days time,charger count,ev other info,ev network,latitude,longitude,date last confirmed,ev connector types,facility type,ev pricing,geom
4,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,US,213-741-1151,Public,5:30am-9pm; pay lot,7.0,,Non-Networked,34.040540,-118.271385,1/10/2023,J1772,PARKING_GARAGE,Free; parking fee,0101000020E6100000000000605E915DC0000000603005...
5,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,US,626-575-6800,Public,24 hours daily,3.0,,Non-Networked,34.068720,-118.064000,9/14/2022,J1772,STATE_GOV,Free,0101000020E6100000000000A018845DC0000000E0CB08...
6,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,US,,Public,24 hours daily,1.0,,Non-Networked,32.899470,-117.243000,1/10/2023,J1772,HOSPITAL,Free; parking fee,0101000020E6100000000000408D4F5DC0000000E02173...
7,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,US,800-256-6219,Public - Call ahead,Dealership business hours; customer use only,2.0,,Non-Networked,34.221664,-118.468370,8/10/2023,J1772,CAR_DEALER,,0101000020E6100000000000C0F99D5DC0000000805F1C...
8,Galleria at Tyler,1299 Galleria at Tyler,Riverside,CA,92503,US,951-351-3110,Public,6am-12am daily,4.0,,Non-Networked,33.909912,-117.459050,9/14/2023,J1772,SHOPPING_MALL,Free,0101000020E610000000000020615D5DC00000000078F4...


In [17]:
list(cleaned_df.columns)

['station name',
 'street address',
 'city',
 'state',
 'zipcode',
 'country',
 'station phone',
 'groups with access code',
 'access days time',
 'charger count',
 'ev other info',
 'ev network',
 'latitude',
 'longitude',
 'date last confirmed',
 'ev connector types',
 'facility type',
 'ev pricing',
 'geom']

In [19]:
selected_columns = cleaned_df[['station name',
 'street address',
 'city',
 'state',
 'zipcode',
 'country',
 'facility type',
 'station phone',
 'latitude',
 'longitude',
 'geom',
 'access days time',
 'charger count',
 'ev network',
 'ev connector types',
 'ev pricing']]
updated_df = selected_columns
updated_df.head()

,station name,street address,city,state,zipcode,country,facility type,station phone,latitude,longitude,geom,access days time,charger count,ev network,ev connector types,ev pricing
4,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,US,PARKING_GARAGE,213-741-1151,34.040540,-118.271385,0101000020E6100000000000605E915DC0000000603005...,5:30am-9pm; pay lot,7.0,Non-Networked,J1772,Free; parking fee
5,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,US,STATE_GOV,626-575-6800,34.068720,-118.064000,0101000020E6100000000000A018845DC0000000E0CB08...,24 hours daily,3.0,Non-Networked,J1772,Free
6,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,US,HOSPITAL,,32.899470,-117.243000,0101000020E6100000000000408D4F5DC0000000E02173...,24 hours daily,1.0,Non-Networked,J1772,Free; parking fee
7,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,US,CAR_DEALER,800-256-6219,34.221664,-118.468370,0101000020E6100000000000C0F99D5DC0000000805F1C...,Dealership business hours; customer use only,2.0,Non-Networked,J1772,
8,Galleria at Tyler,1299 Galleria at Tyler,Riverside,CA,92503,US,SHOPPING_MALL,951-351-3110,33.909912,-117.459050,0101000020E610000000000020615D5DC00000000078F4...,6am-12am daily,4.0,Non-Networked,J1772,Free


In [22]:
# Construct the connection string
db_connection_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Define the schema name and view name
schema_name = 'evChargers'
table_name = 'evcharingstations_all'

# Write the DataFrame as a view to the database
updated_df.to_sql(table_name, con=create_engine(db_connection_str), schema=schema_name, if_exists='replace', index=False)


141